In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn import metrics
import tensorflow as tf
import tensorflow.contrib.slim as slim
from networkx import to_numpy_matrix, karate_club_graph
import networkx as nx

In [2]:
zkc = karate_club_graph()
order = sorted(list(zkc.nodes()))
A = to_numpy_matrix(zkc, nodelist=order)
I = np.eye(zkc.number_of_nodes())          
A_hat = A + I
D_hat = np.array(np.sum(A_hat, axis=0))[0] # 度矩阵
D_hat_ = np.array(np.diag(D_hat))

In [3]:
sess = tf.Session()

dic = {}
dic['Officer'] = 0
dic['Mr. Hi'] = 1
labels = np.zeros((len(zkc), 2))
for i in range(len(zkc)):
    if dic[zkc.nodes[i]['club']] == 0:
        labels[i][0] = 1
        labels[i][1] = 0
    else:
        labels[i][0] = 0
        labels[i][1] = 1

In [4]:
def aggregator(inputs, nodes, neighbors):
    node_feat = tf.nn.embedding_lookup(inputs, nodes)
    neighbors_feat = tf.nn.embedding_lookup(inputs, neighbors)
    
    concat_feat = tf.concat([node_feat, neighbors_feat], axis=1)
    concat_mean = tf.reduce_mean(concat_feat, axis=1)
    
    unit = inputs_features.get_shape().as_list()
    tmp = slim.fully_connected(concat_mean, 1156, activation_fn=tf.nn.relu)
    outputs = tf.reduce_mean(tf.reshape(tmp,[-1,34,34]),axis=0)
    return outputs, tmp

def graphSAGE(inputs, node_inputs, neighbors_inputs, neighbors_num):
    outputs = inputs
    for i in range(neighbors_num):
        neighbors =  neighbors_inputs[i]
        nodes = node_inputs
        outputs, tmp = aggregator(outputs, nodes, neighbors)
        
    outputs_graphSAGE = slim.fully_connected(tmp, 2, activation_fn=tf.nn.softmax)
    return outputs_graphSAGE

inputs_features = tf.placeholder(shape=[34,34],dtype=tf.float32)
inputs_labels = tf.placeholder(shape=[None,2],dtype=tf.float32)
node_inputs = tf.placeholder(shape=[None,1],dtype=tf.int32)
neighbors_inputs = []
for i in range(len(D_hat)):
    neighbors_inputs.append(tf.placeholder(shape=[None,D_hat[i]],dtype=tf.int32, name='tensor'+str(i)))

outputs_graphSAGE = graphSAGE(inputs_features, node_inputs, neighbors_inputs, len(D_hat))

In [20]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=outputs_graphSAGE, labels=inputs_labels))
opt = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

sess.run(tf.global_variables_initializer())

In [21]:
def get_feed_batch():
    I_feed = np.expand_dims([x for x in range(34)],axis=-1)
    feed_dicts = {inputs_features:A_hat, node_inputs:I_feed, inputs_labels:labels}
    
    for i in range(len(neighbors_inputs)):
        y = np.argwhere(A_hat[i] > 0)
        z = []
        for j in range(len(y)):
            z.append(y[j][1])
        feed_dicts[neighbors_inputs[i]] = [np.array(z) for _ in range(34)]
    return feed_dicts

def get_feed(y):
    I_feed = np.expand_dims([y],axis=-1)
    feed_dicts = {inputs_features:A_hat, node_inputs:I_feed, inputs_labels:[labels[y]]}

    for i in range(len(neighbors_inputs)):
        y = np.argwhere(A_hat[i] > 0)
        z = []
        for j in range(len(y)):
            z.append(y[j][1])
        feed_dicts[neighbors_inputs[i]] = [np.array(z)]
    return feed_dicts

In [22]:
for i in range(200):
    """
    for j in range(34):
        feed_dicts = get_feed(j)
        error, _ = sess.run([loss, opt], feed_dict=feed_dicts)
    """
    feed_dicts = get_feed_batch()
    error, _ = sess.run([loss, opt], feed_dict=feed_dicts)
    if i % 50 == 0:
        print('epoch', i, error)
    
print('last error', error)

epoch 0 0.6931472
epoch 50 0.6930295
epoch 100 0.69210947
epoch 150 0.6878026
last error 0.6719878


In [23]:
preds = []
for j in range(34):
    feed_dicts = get_feed(j)
    output = sess.run(outputs_graphSAGE, feed_dict=feed_dicts)[0]
    preds.append(output)

In [24]:
c = 0
for i in range (34):
    if np.argmax(labels[i]) != np.argmax(preds[i]):
        c += 1
        print(preds[i])
print('acc:', 1 - c/34)

acc: 1.0
